In [22]:
import pandas as pd
import csv
from requests import get
import json
from datetime import datetime, timedelta, date
import numpy as np
from scipy.spatial.distance import euclidean, cityblock, cosine
from scipy.stats import pearsonr

import csv
import re
import pandas as pd
import argparse
import collections
import json
import glob
import math
import os
import requests
import string
import sys
import time
import xml
import random

In [45]:
df = pd.read_csv('small_test.csv')
df

,movieId,0331949b45,1030c5a8a9,116dcf4746,1ded6fa4fe
0,32,4.0,5.0,4.0,5.0
1,50,4.0,5.0,NaN,3.5
2,293,5.0,4.5,4.5,4.0
3,344,5.0,4.0,2.5,1.0
4,457,5.0,4.0,3.0,3.0
5,541,5.0,3.5,5.0,3.5
6,586,3.0,3.5,2.0,2.0
7,589,5.0,5.0,4.0,3.5
8,648,4.0,3.0,4.0,2.5
9,1080,2.0,4.5,4.0,3.0


In [48]:
userId = "0331949b45" 

In [49]:
weights = {}
for user in df.columns[1:]:
            if(user!=userId):
                data_subset = df[[userId,user]][df[userId].notnull() & df[user].notnull()]
                dist = euclidean(data_subset[userId],data_subset[user])
                weights[user] = 1.0 / (1.0+dist)
weights

{'1030c5a8a9': 0.10230216299200158,
 '116dcf4746': 0.09811243299910319,
 '1ded6fa4fe': 0.10070203872693176}

In [50]:
df_user = df[['movieId',userId]][df[userId].notnull()]

existing = [tuple(x) for x in df_user.to_records(index=False)]

print(len(existing))

42


In [51]:
df_without = df[df[userId].notnull()].drop(userId,1)

predict = []


/var/folders/05/skbly5dd6ps2gw1n0sqr5s2r0000gp/T/ipykernel_17898/2998811134.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_without = df[df[userId].notnull()].drop(userId,1)


In [52]:
sorted_weights = sorted(weights,key=weights.get, reverse=True)[:1]
sorted_weights

['1030c5a8a9']

In [56]:
for(index,row) in df_without.iterrows():
    pred = 0.0
    weights_sum = 0.0

    for(key) in sorted_weights:
        if not(pd.isna(row.loc[key])):
            pred += weights[key]*row.loc[key]
            weights_sum += weights[key]
            
    if(weights_sum==0):
        pred_ratings=0
    else:
            pred /= weights_sum
    add_tup = (row[0],pred)
    predict.append(add_tup)


predict_dict = dict(predict)
predict


[(32.0, 5.0),
 (50.0, 5.0),
 (293.0, 4.5),
 (344.0, 4.0),
 (457.0, 4.0),
 (541.0, 3.5),
 (586.0, 3.5),
 (589.0, 5.0),
 (648.0, 3.0),
 (1080.0, 4.5),
 (1527.0, 5.0),
 (1682.0, 4.5),
 (1961.0, 4.0),
 (2502.0, 4.0),
 (2542.0, 3.0),
 (3114.0, 4.5),
 (3147.0, 3.0),
 (5218.0, 2.0),
 (5349.0, 2.0),
 (5459.0, 1.5),
 (5952.0, 2.5),
 (5989.0, 4.0),
 (8360.0, 5.0),
 (8368.0, 3.0),
 (8874.0, 3.0),
 (33493.0, 1.5),
 (47610.0, 3.5),
 (48385.0, 2.0),
 (48780.0, 4.5),
 (54286.0, 2.0),
 (59784.0, 4.0),
 (60684.0, 0.0),
 (63082.0, 3.0),
 (68237.0, 3.0),
 (69844.0, 3.0),
 (76093.0, 2.5),
 (78499.0, 4.5),
 (81834.0, 3.0),
 (91500.0, 3.5),
 (99114.0, 4.0),
 (109374.0, 4.5),
 (115713.0, 4.5),
 (32.0, 5.0),
 (50.0, 5.0),
 (293.0, 4.5),
 (344.0, 4.0),
 (457.0, 4.0),
 (541.0, 3.5),
 (586.0, 3.5),
 (589.0, 5.0),
 (648.0, 3.0),
 (1080.0, 4.5),
 (1527.0, 5.0),
 (1682.0, 4.5),
 (1961.0, 4.0),
 (2502.0, 4.0),
 (2542.0, 3.0),
 (3114.0, 4.5),
 (3147.0, 3.0),
 (5218.0, 2.0),
 (5349.0, 2.0),
 (5459.0, 1.5),
 (5952.0, 2

In [54]:
existing

[(32, 4.0),
 (50, 4.0),
 (293, 5.0),
 (344, 5.0),
 (457, 5.0),
 (541, 5.0),
 (586, 3.0),
 (589, 5.0),
 (648, 4.0),
 (1080, 2.0),
 (1527, 5.0),
 (1682, 3.0),
 (1961, 3.0),
 (2502, 3.0),
 (2542, 2.0),
 (3114, 5.0),
 (3147, 4.0),
 (5218, 3.0),
 (5349, 4.0),
 (5459, 2.0),
 (5952, 5.0),
 (5989, 4.0),
 (8360, 4.0),
 (8368, 5.0),
 (8874, 4.0),
 (33493, 4.0),
 (47610, 5.0),
 (48385, 5.0),
 (48780, 5.0),
 (54286, 4.0),
 (59784, 5.0),
 (60684, 4.0),
 (63082, 3.0),
 (68237, 4.0),
 (69844, 5.0),
 (76093, 5.0),
 (78499, 5.0),
 (81834, 5.0),
 (91500, 4.0),
 (99114, 4.0),
 (109374, 5.0),
 (115713, 4.0)]

In [69]:
rmse = 0
ratio = 0.0
non_zero = 0.0

df_user = df[['movieId']]

            
            

rec.array([( 0,     32), ( 1,     50), ( 2,    293), ( 3,    344),
           ( 4,    457), ( 5,    541), ( 6,    586), ( 7,    589),
           ( 8,    648), ( 9,   1080), (10,   1527), (11,   1682),
           (12,   1961), (13,   2502), (14,   2542), (15,   3114),
           (16,   3147), (17,   5218), (18,   5349), (19,   5459),
           (20,   5952), (21,   5989), (22,   8360), (23,   8368),
           (24,   8874), (25,  33493), (26,  47610), (27,  48385),
           (28,  48780), (29,  54286), (30,  59784), (31,  60684),
           (32,  63082), (33,  68237), (34,  69844), (35,  76093),
           (36,  78499), (37,  81834), (38,  91500), (39,  99114),
           (40, 109374), (41, 115713), (42, 134130)],
          dtype=[('index', '<i8'), ('movieId', '<i8')])